In [1]:
#Initialize SQLContext from SparkContext
from pyspark import SparkContext,SQLContext,SparkConf
conf = SparkConf().setAppName("Hello").setMaster("local[2]")
sc = SparkContext(conf=conf)
sqlContext = SQLContext(sc)










In [2]:
#Reading JSON file from HDFS
txns = sqlContext.read.json(
  "/user/gl_faculty2_greatlearning/retail/txnsmall.JSON")
#Display the first 10 records
txns.show(10)

AnalysisException: u'Path does not exist: hdfs://cluster002ns/user/gl_faculty2_greatlearning/retail/txnsmall.JSON;'

In [ ]:
#Flatten the nested structure The line items are nested structure in #each transaction.
#Display the lineitems of first 5 transactions
txns.select("lineItems").take(5)

from pyspark.sql.functions import *

#Explode function in spark sql can be used to flatten the records

txns_new = txns.select( 'txnNo', 'tDate', 'customerNo', 'merchantCity',
                     'state', 'item.category',
                     'item.product', 'item.amount',
                     explode( txns.lineItems ).alias( 'item' ) ).drop( 'item')
txns_new.show(10)


In [ ]:
#Register the new table as temporary ( in memory ) table, so that we can #run SQL Queries on it
txns_new.registerTempTable("txnrecords")

#lets cache the txnrecords table in the memory
sqlContext.cacheTable("txnrecords")

In [ ]:
#Find revenue generated by state and product
revenue_by_state = sqlContext.sql("select state, product, sum( amount ) as total from txnrecords group by state, product")

revenue_by_state.show(10)

#Register the result sets as temporary table
revenue_by_state.registerTempTable('state_revenue')


In [ ]:
#Write an UDF ( User defined function ) to extract week day name from #the date field

import datetime
def getDayOfWeek(date):
  return datetime.datetime.strptime(date, "%m-%d-%Y").strftime('%A')
  

In [ ]:

from pyspark.sql.types import StringType
sqlContext.udf.register("getDayOfWeek", lambda date: getDayOfWeek(date),
                      StringType() )
					  
revenue_by_state = sqlContext.sql( '''select weekday as weekday, 
                                  round( sum( amount ), 2 ) as total 
                                  from ( select getDayOfWeek( tDate ) as weekday, 
                                  amount from txnrecords ) txns 
                                  group by weekday order by total desc''' )

revenue_by_state.show(10) 

In [8]:
#Initialize SQLContext from SparkContext
from pyspark import SparkContext,SQLContext,SparkConf
conf = SparkConf().setAppName("Hello").setMaster("local[2]")
#sc = SparkContext(conf=conf)
#sqlContext = SQLContext(sc)

#Reading JSON file from HDFS
txns = sqlContext.read.json(
  "/user/gl_faculty2_greatlearning/retail/txnsmall.JSON")
#Display the first 10 records
#txns.show(10)

#Flatten the nested structure The line items are nested structure in #each transaction.
#Display the lineitems of first 5 transactions
#txns.select("lineItems").take(5)

from pyspark.sql.functions import *

txns_new = txns.select( 'txnNo', 'tDate', 'customerNo', 'merchantCity',
                     'state', 'item.category',
                     'item.product', 'item.amount',
                     explode( txns.lineItems ).alias( 'item' ) ).drop( 'item')


#Register the new table as temporary ( in memory ) table, so that we can #run SQL Queries on it
txns_new.registerTempTable("txnrecords")

#lets cache the txnrecords table in the memory
sqlContext.cacheTable("txnrecords")


#Find revenue generated by state and product
revenue_by_state = sqlContext.sql("select state, product, sum( amount ) as total from txnrecords group by state, product")

import datetime
def getDayOfWeek(date):
  return datetime.datetime.strptime(date, "%m-%d-%Y").strftime('%A')
  
from pyspark.sql.types import StringType
sqlContext.udf.register("getDayOfWeek", lambda date: getDayOfWeek(date),
                      StringType() )
					  
revenue_by_state = sqlContext.sql( '''select weekday as weekday, 
                                  round( sum( amount ), 2 ) as total 
                                  from ( select getDayOfWeek( tDate ) as weekday, 
                                  amount from txnrecords ) txns 
                                  group by weekday order by total desc''' )

revenue_by_state.show(10) 



+---------+--------+
|  weekday|   total|
+---------+--------+
| Thursday| 94549.2|
|Wednesday|85091.56|
|   Monday|81712.77|
|   Sunday|79634.08|
|  Tuesday|79594.51|
| Saturday|78114.84|
|   Friday| 71809.1|
+---------+--------+

